In [18]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import re
import gc
import pickle
from tqdm import tqdm
from itertools import product
from string import punctuation

In [20]:
# data folder
DATA_FOLDER = "/content/drive/My Drive/Predict Future Sales"

In [21]:
# the aggregated training + test set. 
dfAggregatedTrainTest = pd.read_csv(f'{DATA_FOLDER}/matrix.csv', sep='\t')

In [22]:
# print information about the index dtype and columns, non-null values and memory usage
dfAggregatedTrainTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11128004 entries, 0 to 11128003
Data columns (total 13 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   shop_id                    int64  
 1   item_id                    int64  
 2   date_block_num             int64  
 3   target                     float64
 4   revenue                    float64
 5   target_shop                float64
 6   revenue_per_shop           float64
 7   target_item                float64
 8   revenue_per_item           float64
 9   item_category_id           int64  
 10  target_item_category       float64
 11  revenue_per_item_category  float64
 12  target_item_price          float64
dtypes: float64(9), int64(4)
memory usage: 1.1 GB


In [23]:
"""
    Function name: Downcastdtypes
    
    Objective: Change column types in the dataframe
    
    Summary algorithmic description: `float64` type to `float32`
                                     `int64`   type to `int32`
    
    Input parameters: dfData: dataframe
    
    Return : downcasted dataframe
    
    Date : 23/12/2021
    
    Coding: INSA CVL - Van Tuan BUI  
"""
def Downcastdtypes(dfData):
    # Select columns to downcast
    # floating columns
    clFloatColumn = [sColumn for sColumn in dfData if dfData[sColumn].dtype == "float64"]
    # integer columns
    clIntColumn = [sColumn for sColumn in dfData if dfData[sColumn].dtype == "int64"]
    
    # Downcast
    # `float64` type to `float32`
    dfData[clFloatColumn] = dfData[clFloatColumn].astype(np.float32)
    # `int64`   type to `int32`
    dfData[clIntColumn]   = dfData[clIntColumn].astype(np.int32)
    # Return downcasted dataframe
    return dfData

In [24]:
# Downcast dtypes from 64 to 32 bit to save memory
dfAggregatedTrainTest = Downcastdtypes(dfAggregatedTrainTest)
# print information about the index dtype and columns, non-null values and memory usage
dfAggregatedTrainTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11128004 entries, 0 to 11128003
Data columns (total 13 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   shop_id                    int32  
 1   item_id                    int32  
 2   date_block_num             int32  
 3   target                     float32
 4   revenue                    float32
 5   target_shop                float32
 6   revenue_per_shop           float32
 7   target_item                float32
 8   revenue_per_item           float32
 9   item_category_id           int32  
 10  target_item_category       float32
 11  revenue_per_item_category  float32
 12  target_item_price          float32
dtypes: float32(9), int32(4)
memory usage: 551.8 MB


In [25]:
# Display first 5 rows in the data
dfAggregatedTrainTest.head()

,shop_id,item_id,date_block_num,target,revenue,target_shop,revenue_per_shop,target_item,revenue_per_item,item_category_id,target_item_category,revenue_per_item_category,target_item_price
0,0,19,0,0.0,0.0,5578.0,2966412.0,1.0,28.000000,40,33532.0,8678094.0,28.000000
1,0,27,0,0.0,0.0,5578.0,2966412.0,7.0,16275.000000,19,9024.0,14162480.0,2325.000000
2,0,28,0,0.0,0.0,5578.0,2966412.0,8.0,4392.000000,30,22261.0,10032387.0,549.000000
3,0,29,0,0.0,0.0,5578.0,2966412.0,5.0,11886.000000,23,4172.0,7709925.5,2377.199951
4,0,32,0,6.0,1326.0,5578.0,2966412.0,299.0,101515.859375,40,33532.0,8678094.0,339.517914


In [26]:
# index columns
INDEX_COLS = ['shop_id', 'item_id', 'date_block_num']
# data columns
DATA_COLS = ['item_price', 'item_cnt_day']

In [27]:
# Calculate sum of the target values using all rows from zero to n minus 1
dsCumSum = dfAggregatedTrainTest.groupby('item_category_id')['target'].cumsum() - dfAggregatedTrainTest['target']
# (n-1)th row
dsCumCount = dfAggregatedTrainTest.groupby('item_category_id').cumcount()
# mean encoding from zero to n minus one 
dfAggregatedTrainTest['item_category_id_target_enc'] = dsCumSum / dsCumCount
# Global mean of target
fGlobalMean = dfAggregatedTrainTest['target'].mean()
# Fill Nans with global mean
dfAggregatedTrainTest['item_category_id_target_enc'].fillna(fGlobalMean, inplace=True) 
# Print correlation
caEncodedItemFeature = dfAggregatedTrainTest['item_category_id_target_enc'].values
fCorr = np.corrcoef(dfAggregatedTrainTest['target'].values, caEncodedItemFeature)[0][1]
print(fCorr)

0.3841296007441824


In [28]:
# Display first 5 rows in the data
dfAggregatedTrainTest.head()

,shop_id,item_id,date_block_num,target,revenue,target_shop,revenue_per_shop,target_item,revenue_per_item,item_category_id,target_item_category,revenue_per_item_category,target_item_price,item_category_id_target_enc
0,0,19,0,0.0,0.0,5578.0,2966412.0,1.0,28.000000,40,33532.0,8678094.0,28.000000,0.328518
1,0,27,0,0.0,0.0,5578.0,2966412.0,7.0,16275.000000,19,9024.0,14162480.0,2325.000000,0.328518
2,0,28,0,0.0,0.0,5578.0,2966412.0,8.0,4392.000000,30,22261.0,10032387.0,549.000000,0.328518
3,0,29,0,0.0,0.0,5578.0,2966412.0,5.0,11886.000000,23,4172.0,7709925.5,2377.199951,0.328518
4,0,32,0,6.0,1326.0,5578.0,2966412.0,299.0,101515.859375,40,33532.0,8678094.0,339.517914,0.000000


In [29]:
# True target values are clipped into [0,20] range.
TARGET_RANGE = [0, 20]
# clip the target into [0,20] range
# dfAggregatedTrainTest['target'] = dfAggregatedTrainTest['target'].clip(*TARGET_RANGE)
# Remove 
dfAggregatedTrainTest = dfAggregatedTrainTest[dfAggregatedTrainTest['target'] <= 20]

In [30]:
# List of columns that we will use to create lags
COLS_TO_LAG = ['target', 'target_shop', 'target_item', 'target_item_category', 'target_item_price'] 
# shift lag range
SHIFT_RANGE = [1, 2, 3, 4, 5, 12]

In [31]:
# Loop over all month shifts
for iMonthShift in SHIFT_RANGE:
    # shifted aggregated training and test data
    dfAggregatedTrainTestShift = dfAggregatedTrainTest[INDEX_COLS + COLS_TO_LAG].copy()
    dfAggregatedTrainTestShift['date_block_num'] = dfAggregatedTrainTestShift['date_block_num'] + iMonthShift
    # new lagged columns
    oColumnLambda = lambda sColumn: '{}_lag_{}'.format(sColumn, iMonthShift) if sColumn in COLS_TO_LAG else sColumn
    dfAggregatedTrainTestShift = dfAggregatedTrainTestShift.rename(columns=oColumnLambda)
    # Join lagged columns
    dfAggregatedTrainTest = dfAggregatedTrainTest.merge(dfAggregatedTrainTestShift, on=INDEX_COLS, how='left').fillna(0)
    # Delete the shifted aggregated training and test data, but it doesn't release memory
    del dfAggregatedTrainTestShift
    # release the RAM/memory
    gc.collect()

In [32]:
# Display first 5 rows in the data
dfAggregatedTrainTest.head()

,shop_id,item_id,date_block_num,target,revenue,target_shop,revenue_per_shop,target_item,revenue_per_item,item_category_id,target_item_category,revenue_per_item_category,target_item_price,item_category_id_target_enc,target_lag_1,target_shop_lag_1,target_item_lag_1,target_item_category_lag_1,target_item_price_lag_1,target_lag_2,target_shop_lag_2,target_item_lag_2,target_item_category_lag_2,target_item_price_lag_2,target_lag_3,target_shop_lag_3,target_item_lag_3,target_item_category_lag_3,target_item_price_lag_3,target_lag_4,target_shop_lag_4,target_item_lag_4,target_item_category_lag_4,target_item_price_lag_4,target_lag_5,target_shop_lag_5,target_item_lag_5,target_item_category_lag_5,target_item_price_lag_5,target_lag_12,target_shop_lag_12,target_item_lag_12,target_item_category_lag_12,target_item_price_lag_12
0,0,19,0,0.0,0.0,5578.0,2966412.0,1.0,28.000000,40,33532.0,8678094.0,28.000000,0.328518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,27,0,0.0,0.0,5578.0,2966412.0,7.0,16275.000000,19,9024.0,14162480.0,2325.000000,0.328518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,28,0,0.0,0.0,5578.0,2966412.0,8.0,4392.000000,30,22261.0,10032387.0,549.000000,0.328518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,29,0,0.0,0.0,5578.0,2966412.0,5.0,11886.000000,23,4172.0,7709925.5,2377.199951,0.328518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,32,0,6.0,1326.0,5578.0,2966412.0,299.0,101515.859375,40,33532.0,8678094.0,339.517914,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Don't use old data from year 2013
dfAggregatedTrainTest = dfAggregatedTrainTest[dfAggregatedTrainTest['date_block_num'] >= 12]
# print information about the index dtype and columns, non-null values and memory usage
dfAggregatedTrainTest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6633099 entries, 4483430 to 11116528
Data columns (total 44 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   shop_id                      int32  
 1   item_id                      int32  
 2   date_block_num               int32  
 3   target                       float32
 4   revenue                      float32
 5   target_shop                  float32
 6   revenue_per_shop             float32
 7   target_item                  float32
 8   revenue_per_item             float32
 9   item_category_id             int32  
 10  target_item_category         float32
 11  revenue_per_item_category    float32
 12  target_item_price            float32
 13  item_category_id_target_enc  float32
 14  target_lag_1                 float32
 15  target_shop_lag_1            float32
 16  target_item_lag_1            float32
 17  target_item_category_lag_1   float32
 18  target_item_price_lag_1      float3

In [34]:
# Save features
dfAggregatedTrainTest.to_hdf(f'{DATA_FOLDER}/features.h5', key='df', mode='w', complib='zlib') 